In [ ]:
import pandas as pd
import json
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
import csv
import plotly.plotly as py
from dash.dependencies import Input, Output
from datetime import datetime as dt
import datetime

In [ ]:
with open('events.json') as f:
    events = json.load(f) # events ist dann die Liste mit Dictionaries

for event in events[:10]:  # Print only the top 10 (HBü)
    print(event)

In [ ]:
df = pd.DataFrame(events)
df.head(10) # Print only top 10 (HBü)

In [ ]:
# Auskommentiert, du verwendet das später nicht. (HBü)
# counttimestamp = df['timestamp'].value_counts()
# print (counttimestamp)

In [ ]:
df['timestamp']=pd.to_datetime(df.timestamp)
df.dtypes

In [ ]:
df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['week'] = df['timestamp'].dt.week
df['weekday'] = df['timestamp'].dt.weekday_name
df['day'] = df['timestamp'].dt.day
df['just_date'] = df['timestamp'].dt.date
# df['counttimestamp'] = df['timestamp'].value_counts()  # Auskommentiert. Das kann raus, verwendest du nie.  (HBü)
df.head(10) # Print only top 10  (HBü)

In [ ]:
# df['month'].value_counts()   # Auskommentiert. Schadet natürlich nicht, aber am Ende würde ich es rausnehmen (HBü)
# df['minute'].value_counts()

In [ ]:
df['event_type'].value_counts()

In [ ]:
# Auskommentiert, weil macht keinen Sinn. "frame", "face" waren nur Beispielwerte, in real wissen wir ja nicht, was kommt. (HBü)
# frame=df.event_type.str.contains(r'frame').sum()
# face=df.event_type.str.contains(r'face').sum() 
# movement=df.event_type.str.contains(r'movement').sum()
# print (frame)
# print (face)
# print (movement)

In [ ]:
# Auskommentiert. Schadet natürlich nicht, aber am Ende würde ich es rausnehmen (HBü)
# test=df['Anzahl_ID'].sum()
# print(test)

In [ ]:
df.dtypes

In [ ]:
#start = datetime.datetime(2018, 1, 1 )
#end = datetime.datetime.now

available_camera_indicators = df['camera_id'].unique() 
av_camera_options=[{'value': i, 'label': i} for i in available_camera_indicators]

available_event_indicators = df['event_type'].unique() 
av_event_options=[{'value': i, 'label': i} for i in available_event_indicators]


# %tb   # Auskommentiert. Macht wohl Traceback https://ipython.readthedocs.io/en/stable/interactive/magics.html . Brauchen wir hier nicht. (HBü)
app = dash.Dash()
app.scripts.config.serve_locally=True  # Hinzugefügt. Verhindert, das Dash komische Sachen aus dem Internet lädt. (HBü)

app.layout = html.Div(children=[
    html.H1(children='Test Report', style= {'margin':'45px 0px 90px 50px'}),  # Geändert in Überschrift. (HBü)
    
    html.Div([   # Die Dropdown nochmal in ein DIV gepackt (HBü)
        html.P('Filter Dataset:', style={'margin':'0px 0px 5px 0px'}),
        dcc.Dropdown(
            id='camera-dropdown',
            options=av_camera_options
        ),   

        dcc.Dropdown(
            id='event-dropdown',
            options=av_event_options
        ),

        dcc.Dropdown(
            id='xaxis-dropdown',
            options=[
                {'label': 'minute', 'value': 'minute'},
                {'label': 'weekday', 'value': 'weekday'},
                {'label': 'hour', 'value': 'hour'}
            ],
            value='minute'
        ),
    ], style={          # Styling für die Dropdowns. Experimentiere gerne damit rum! (HBü)
            'width': '170px',
            'position': 'absolute',
            'right': '10px',
            'top': '10px',
            'z-index': 10,
            'background-color': '#f7f7f7',
            'padding': '10px',
            'border-radius': '13px',
            'box-shadow': '1px 2px 5px rgba(0,0,0,0.5)'
        }),
    
    html.Div([
        html.Div([   # Sortiere die Charts ein bisschen um... (HBü)
            # html.H3('column2'),  # Auskommentiert. Titel im Chart reichen imho. (HBü)
            dcc.Graph(
                id='camera-graph-line',
                #style={'width': '45%', 'height': '400px'},  # Auskommentiert, brauch man mit der Flex-Box-Methode nicht. (HBü)
                figure={
                    'data': [{'x':'df.minute',    # Ich schreib sowas meist untereinander, erhöht m.E. die Lesbarkeit (HBü)
                             'y':'df.timestamp.count()', # Count, statt Anzahl_id (HBü)
                             'text':'df.event_type', 
                             'type':'line', 
                             'name': 'IDK', 
                             'mode':'lines',
                             'marker': {'size': 5}
                             }]
                }
            ), 
        ], style={'flex': '1'}),     # "Flexbox" Methode. Die habe ich gerad erst gegoogelt. (HBü)         
        html.Div([
            #html.H3('column1'),  # Auskommentiert. Titel im Chart reichen imho. (HBü)
            dcc.Graph(
                id='camera-graph-histogram',
                #style={'width': '45%', 'height': '400px'},  # Auskommentiert, brauch man mit der Flex-Box-Methode nicht. (HBü)
                figure={
                    'data': [{'x':'df.minute',    # Ich schreib sowas meist untereinander, erhöht m.E. die Lesbarkeit (HBü)
                             'y':'df.timestamp.count()',# Count, statt Anzahl_id (HBü)
                             'text':'df.event_type', 
                             'type':'histogram', 
                             'name' : 'IDK', 
                             'mode':'markers', 
                             'className':'col s12 m6 l6'
                            }]
                }

            ),
        ], style={'flex': '1'}),     # "Flexbox" Methode (HBü)
        
        html.Div([
            #html.H3('column3'),  # Auskommentiert. Titel im Chart reichen imho. (HBü)
            dcc.Graph(
                id='camera-graph-pie',
                #style={'width': '60%', 'height': '400px'},  # Auskommentiert, brauch man mit der Flex-Box-Methode nicht. (HBü)
                figure={
                    'data': [
                            {'values':[    # Diese Initialen Werte stimmen natürlich nicht mehr. Ändern überlass ich dir... ;-) HBü
                                df.event_type.str.contains(r'frame').sum(), 
                                df.event_type.str.contains(r'face').sum(), 
                                df.event_type.str.contains(r'movement').sum()
                            ], 'labels':['frame', 'face','movement'], 'type':'pie'}

                    ]
                }
            ),
        ], style={'flex': '1'})     # "Flexbox" Methode (HBü)  
    ], style={'display': 'flex'})   # "Flexbox" Methode (HBü)
    

])
    

# In graph1 umbenannt. Vorsicht, zwei Funktionen dürfen nicht gleich heißen! (HBü)
@app.callback(
    Output('camera-graph-histogram', 'figure'),
    [Input('event-dropdown', 'value'),
     Input('camera-dropdown', 'value'),
     Input('xaxis-dropdown', 'value')])
def update_graph1(selected_dropdown_value1, selected_dropdown_value2, selected_dropdown_value3):
    dfh =df
    if selected_dropdown_value1 != None:
        dfhh = dfh[dfh.event_type == selected_dropdown_value1]
    else:dfhh = dfh
    if selected_dropdown_value2 != None:
        dfhhh = dfhh[dfh.camera_id == selected_dropdown_value2]
    else: dfhhh = dfhh
    return{
        'data':[go.Scatter(
            x=dfhhh[selected_dropdown_value3],
            y=dfhhh['timestamp'].count(),  # Count, statt Anzahl_id (HBü)
            text=dfhhh['event_type'],
            name= 'IDK',
            type='histogram',
            mode='markers')
        ],
            'layout': {
                'title' : 'Reporting cameras'
            }
    }
    
@app.callback(
    Output('camera-graph-pie', 'figure'),
    [Input('camera-dropdown', 'value')])
def update_graph2(selected_dropdown_value2):
    dfp = df
    if selected_dropdown_value2 != None:
        dfpp = dfp[dfp.camera_id == selected_dropdown_value2]
    else: dfpp=dfp
    
    # Auskommentiert. Das müssen wir jetzt dynamisch machen, weil wir nicht nur die drei Werte haben (HBü)
    # frame2 = dfpp.event_type.str.contains(r'frame').sum()
    # face2 = dfpp.event_type.str.contains(r'face').sum()
    # movement2 = dfpp.event_type.str.contains(r'movement').sum()
    
    # Stattdessen gruppieren wir nach "event_type" und zählen die Vorkommen: (HBü)
    df_event_types = dfpp.groupby('event_type').count()
    
    return{
        'data':[go.Pie(
            values=df_event_types.iloc[:,0],      # Nimm die erste Spalte als Werte (HBü)
            labels=df_event_types.index.values,   # Und die Werte im Index als Beschriftung (HBü)
            text=dfpp['event_type'],
            name= 'IDK',
            hole= .4,
            type='pie')
        ],
            'layout': {
                'title' : 'Reporting cameras-Pie'
            }
    } 


@app.callback(
    Output('camera-graph-line', 'figure'),
    [Input('event-dropdown', 'value'),
     Input('camera-dropdown', 'value')])
def update_graph3(selected_dropdown_value1, selected_dropdown_value2):  # In graph3 umbenannt. Vorsicht, zwei Funktionen dürfen nicht gleich heißen! (HBü)
    dfh =df
    if selected_dropdown_value1 != None:
        dfhh = dfh[dfh.event_type == selected_dropdown_value1]
    else:dfhh = dfh
    if selected_dropdown_value2 != None:
        dfhhh = dfhh[dfh.camera_id == selected_dropdown_value2]
    else: dfhhh = dfhh
        
    # Das mit der Zeitreihe müssen wir anders machen: (HBü)
    # Erst brauchen wir ein Datetime-Objekt: 
    dfhhh['datetime'] = pd.to_datetime(dfhhh['timestamp'], format='%d%b%Y %H:%M:%S.%f')
    # Das muss dann als index verwendet werden:
    dfhhh = dfhhh.set_index('datetime')
    # Jetzt können wir ganz einfach auf verschiedene Zeitspannen aggregieren:
    df_resampled = dfhhh.resample('15Min').count()   # D=Tag, H=Stunde, 10min=10 Minuten
    
    # print(df_resampled)  # Kannste dir so mal ausgeben lassen, damit weißt, wie es aussieht...

    return{
        'data':[go.Scatter(
            x=df_resampled.index,          # Jetzt den Zeit-Index für die X-Achse  nehmen (HBü)
            y=df_resampled['timestamp'],   # und irgendwas anderes für die Y-Achse
            text=dfhhh['event_type'],
            name= 'IDK',
            type='line',
            mode='lines+markers')
        ],
            'layout': {
                'title' : 'Reporting cameras line'
            }
    }



if __name__ == '__main__':
    app.run_server(debug=False)